# Clustering in Latent Space for Multiclass Data

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), ".."))

In [ ]:
from pyldl.activations import Sigmoid, Softmax
from pyldl.layers import AutoEncoder, Sequential
from pyldl.losses import BCELoss
from pyldl.modules import Linear
from pyldl.optimizers import SGD
from pyldl.utils import encode_one_hot

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

## Dataset and Network

In [ ]:
np.random.seed(0)

In [ ]:
digits = load_digits()

X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train_one_hot = encode_one_hot(y_train)
y_test_one_hot = encode_one_hot(y_test)

In [ ]:
network = AutoEncoder(Sequential(Linear(X_train.shape[1], 128), Sigmoid(), Linear(128, 64), Sigmoid(), Linear(64, y_train_one_hot.shape[1]), Softmax()), Sequential(Linear(y_train_one_hot.shape[1], 64), Sigmoid(), Linear(64, 128), Sigmoid(), Linear(128, X_train.shape[1]), Sigmoid()))
loss = BCELoss()

## Clustering